In [29]:
from external_potential import Framework, PotentialType
from dft_ad_jax import *
from si_units import *
from glob import glob
import numpy as np
from feos.dft import *
from feos.pcsaft import PcSaftParameters as PcSaftParametersFeos
import jax.numpy as jnp
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import traceback
import os
import csv
from datetime import datetime
import json


In [30]:
def generate_framework(cif_file):#, rc, ngrid, solid_parameters, fluid_parameters):
    return Framework(
        cif=cif_file,
        #cutoff_radius=rc,
        #ngrid=ngrid,
        #solid_parameters=solid_parameters,
        #fluid_parameters=fluid_parameters,
        #maximum_reduced_energy=10,
        #jax_enable_x64=True
    )

In [31]:
def chem_res_pot_bulk(state, dft):
    density_bulk = jnp.array(state.partial_density * NAV * ANGSTROM**3)
    density_bulk_dim = density_bulk[:, None, None, None] * jnp.ones([len(components)] + grid.n_grid)
    return dft.df_drho(density_bulk_dim)[:, 0, 0, 0]

In [32]:
#cif_files = glob("/Users/danielbock/MOF_CIF/*.cif")
cif_files = glob("/Users/danielbock/VSCODE/MA/structures/*.cif")

failed_structures = []

pressures=[1*BAR] #np.linspace(0.1, 100, 5) * BAR
temperatures=[400*KELVIN] #np.linspace(300, 500, 5) * KELVIN
ngrid = 32

with open("dreiding.json", "r") as file:
    solid_parameters = json.load(file)

components = ['methane']
fluid_parameters = {
    "methane": {"m": 1.0, "sigma": 3.70051, "epsilon_k": 150.07147},
}
fluid_file = 'parameters/pcsaft/esper2023.json'
pcsaft_params = pcsaft_from_file(components, fluid_file)
params_feos = PcSaftParametersFeos.from_json(components, fluid_file)
func = HelmholtzEnergyFunctional.pcsaft(params_feos, FMTVersion.AntiSymWhiteBear)
rc = 12.0
"""
for cif_file in cif_files:
    framework = generate_framework(cif_file)#, rc, ngrid, solid_parameters, fluid_parameters)
    grid = Grid(
        n_grid=[ngrid]*3,  # 3D Grid definieren
        length=framework.unitcell.lattice.lengths,
        skew_angles=framework.unitcell.lattice.angles,
        lanczos_sigma=True
    )

    for temperature in temperatures: 
        dft  = DFT(grid, pcsaft_params, temperature / KELVIN)
        for pressure in pressures:
            state = State(func, temperature=temperature, pressure=pressure)
            chem_res_pot_bulky = chem_res_pot_bulk(state, dft)
            #print(chem_res_pot_bulky.shape)"""

'\nfor cif_file in cif_files:\n    framework = generate_framework(cif_file)#, rc, ngrid, solid_parameters, fluid_parameters)\n    grid = Grid(\n        n_grid=[ngrid]*3,  # 3D Grid definieren\n        length=framework.unitcell.lattice.lengths,\n        skew_angles=framework.unitcell.lattice.angles,\n        lanczos_sigma=True\n    )\n\n    for temperature in temperatures: \n        dft  = DFT(grid, pcsaft_params, temperature / KELVIN)\n        for pressure in pressures:\n            state = State(func, temperature=temperature, pressure=pressure)\n            chem_res_pot_bulky = chem_res_pot_bulk(state, dft)\n            #print(chem_res_pot_bulky.shape)'

In [35]:
def temp_pressure(file1='/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/dft_fckin_clean_kond_64grid.csv', file2='/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/DFT_Data_clean_06_10.csv'):
    # CSV-Dateien einlesen
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)    
    # Nur relevante Spalten behalten
    cols = ["temperature_kelvin", "pressure_bar"]
    df1 = df1[cols]
    df2 = df2[cols]
    # Zusammenführen
    combined = pd.concat([df1, df2], ignore_index=True)
    # Doppelte Kombinationen entfernen
    combined_unique = combined.drop_duplicates(subset=cols)
    # Einzigartige Werte je Spalte extrahieren und sortieren
    temperature_values = sorted(combined_unique["temperature_kelvin"].unique())
    pressure_values = sorted(combined_unique["pressure_bar"].unique())
    
    # Mit Einheiten versehen
    temperatures = [t * KELVIN for t in temperature_values]
    pressures = [p * BAR for p in pressure_values]
    
    return temperatures, pressures

In [36]:
cif_files = glob("/Users/danielbock/VSCODE/MA/structures/*.cif")
output_csv = "/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/bulk_potentials.csv"

temperatures, pressures = temp_pressure()
print(temperatures)
print(pressures)

#pressures = [1 * BAR]
#temperatures = [400 * KELVIN]

ngrid = 32

solid_parameters = {
    "O": { "sigma": 3.21, "epsilon_k": 76.3, "k_si": 0.0},
    "Si": { "sigma": 3.3, "epsilon_k": 22.0, "k_si": 0.0 },
}

components = ['methane']
fluid_file = 'parameters/pcsaft/esper2023.json'
pcsaft_params = pcsaft_from_file(components, fluid_file)
params_feos = PcSaftParametersFeos.from_json(components, fluid_file)
func = HelmholtzEnergyFunctional.pcsaft(params_feos)
failed_structures = []

# ---------- Daten sammeln ----------
rows = []
for cif_file in cif_files:
    structure_name = Path(cif_file).stem
    print(f"Verarbeite Struktur: {structure_name}")

    try:
        framework = generate_framework(cif_file)

        grid = Grid(
            n_grid=[ngrid] * 3,
            length=framework.unitcell.lattice.lengths,
            skew_angles=framework.unitcell.lattice.angles,
            lanczos_sigma=True
        )

        for temperature in temperatures:
            dft = DFT(grid, pcsaft_params, float(temperature / KELVIN))

            for pressure in pressures:
                state = State(func, temperature=temperature, pressure=pressure)
                mu_bulk_vec = chem_res_pot_bulk(state, dft)  # shape (1,)
                potential = np.asarray(mu_bulk_vec).item()

                rows.append({
                    "structure_name": structure_name,
                    "temperature_kelvin": float(temperature / KELVIN),
                    "pressure_bar": float(pressure / BAR),
                    "chem_potential_bulk": potential
                })

    except Exception as e:
        failed_structures.append({"structure_name": structure_name, "error": str(e)})
        print(f"Fehler bei {structure_name}: {e}")



[300 K, 325 K, 350 K, 375 K, 400 K, 425 K, 450 K, 475 K, 500 K, 525 K, 550 K, 575 K, 600 K, 625 K, 650 K, 675 K, 700 K, 725 K, 750 K]
[10 kPa, 100 kPa, 807.1428571428571 kPa, 1.5142857142857142 MPa, 2.2214285714285715 MPa, 2.928571428571429 MPa, 3.6357142857142843 MPa, 4.3428571428571425 MPa, 5.05 MPa, 5.757142857142857 MPa, 6.4642857142857135 MPa, 7.171428571428571 MPa, 7.878571428571428 MPa, 8.585714285714285 MPa, 9.292857142857144 MPa, 10 MPa]
Verarbeite Struktur: JSY


/Users/danielbock/MASTERTHESIS/MASTA/.venv/lib/python3.11/site-packages/pymatgen/core/structure.py:3109: UserWarning: Issues encountered while parsing CIF: Skipping relative stoichiometry check because CIF does not contain formula keys.
  struct = parser.parse_structures(primitive=primitive)[0]


Verarbeite Struktur: MWW


/Users/danielbock/MASTERTHESIS/MASTA/.venv/lib/python3.11/site-packages/pymatgen/core/structure.py:3109: UserWarning: Issues encountered while parsing CIF: 10 fractional coordinates rounded to ideal values to avoid issues with finite precision.
Skipping relative stoichiometry check because CIF does not contain formula keys.
  struct = parser.parse_structures(primitive=primitive)[0]


Verarbeite Struktur: JSN
Verarbeite Struktur: ANO
Verarbeite Struktur: ATV
Verarbeite Struktur: MAZ
Verarbeite Struktur: IWS
Verarbeite Struktur: ITH
Verarbeite Struktur: PUN
Verarbeite Struktur: ETR
Verarbeite Struktur: RTH
Verarbeite Struktur: GME
Verarbeite Struktur: RWR
Verarbeite Struktur: EON
Verarbeite Struktur: IWR
Verarbeite Struktur: SEW
Verarbeite Struktur: SSO
Verarbeite Struktur: OBW
Verarbeite Struktur: HEU
Verarbeite Struktur: SFN
Verarbeite Struktur: DFT
Verarbeite Struktur: MOR
Verarbeite Struktur: SIV
Verarbeite Struktur: NSI
Verarbeite Struktur: BPH


...

/Users/danielbock/MASTERTHESIS/MASTA/.venv/lib/python3.11/site-packages/pymatgen/core/structure.py:3109: UserWarning: Issues encountered while parsing CIF: 2 fractional coordinates rounded to ideal values to avoid issues with finite precision.
Skipping relative stoichiometry check because CIF does not contain formula keys.
  struct = parser.parse_structures(primitive=primitive)[0]


Verarbeite Struktur: SBE
Verarbeite Struktur: MER
Verarbeite Struktur: MFI
Verarbeite Struktur: YUG
Verarbeite Struktur: UOS
Verarbeite Struktur: LTL
Verarbeite Struktur: ESV
Verarbeite Struktur: EDI
Verarbeite Struktur: DON
Verarbeite Struktur: CDO
Verarbeite Struktur: SBS
Verarbeite Struktur: SVV
Verarbeite Struktur: STI
Verarbeite Struktur: AFR
Verarbeite Struktur: AEI
Verarbeite Struktur: AFV
Verarbeite Struktur: SVR
Verarbeite Struktur: OSO
Verarbeite Struktur: HZF
Verarbeite Struktur: EEI
Verarbeite Struktur: UOV
Verarbeite Struktur: EPI
Verarbeite Struktur: BOZ
Verarbeite Struktur: ASV
Verarbeite Struktur: CHI
Verarbeite Struktur: SOF
Verarbeite Struktur: CGF
Verarbeite Struktur: FAU
Verarbeite Struktur: AEL
Verarbeite Struktur: SZR
Verarbeite Struktur: AEN
Verarbeite Struktur: VET
Verarbeite Struktur: CSV
Verarbeite Struktur: CGS
Verarbeite Struktur: SAO
Verarbeite Struktur: SBT
Verarbeite Struktur: SOD
Verarbeite Struktur: AST
Verarbeite Struktur: SOS
Verarbeite Struktur: IRN


/Users/danielbock/MASTERTHESIS/MASTA/.venv/lib/python3.11/site-packages/pymatgen/core/structure.py:3109: UserWarning: Issues encountered while parsing CIF: 6 fractional coordinates rounded to ideal values to avoid issues with finite precision.
Skipping relative stoichiometry check because CIF does not contain formula keys.
  struct = parser.parse_structures(primitive=primitive)[0]


Verarbeite Struktur: RWY
Verarbeite Struktur: WEN
Verarbeite Struktur: EOS
Verarbeite Struktur: ITT
Verarbeite Struktur: SFF
Verarbeite Struktur: MOZ
Verarbeite Struktur: VNI
Verarbeite Struktur: JSR
Verarbeite Struktur: FER
Verarbeite Struktur: MTT
Verarbeite Struktur: CZP
Verarbeite Struktur: DFO
Verarbeite Struktur: ATN
Verarbeite Struktur: ITG
Verarbeite Struktur: EWF
Verarbeite Struktur: YFI
Verarbeite Struktur: RUT
Verarbeite Struktur: EUO
Verarbeite Struktur: PON
Verarbeite Struktur: PAR
Verarbeite Struktur: UEI
Verarbeite Struktur: ATO
Verarbeite Struktur: ANA
Verarbeite Struktur: JSW
Verarbeite Struktur: MTW
Verarbeite Struktur: ABW
Verarbeite Struktur: CAN
Verarbeite Struktur: AVE
Verarbeite Struktur: PTF
Verarbeite Struktur: RON
Verarbeite Struktur: ION
Verarbeite Struktur: IMF
Verarbeite Struktur: WEI
Verarbeite Struktur: EWS
Verarbeite Struktur: RTE
Verarbeite Struktur: EZT
Verarbeite Struktur: ITE
Verarbeite Struktur: ITR
Verarbeite Struktur: AWW
Verarbeite Struktur: DDR


ValueError: dict contains fields not in fieldnames: 'chem_potential_bulk'

In [38]:
# ---------- CSV schreiben ----------
with open(output_csv, "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=["structure_name", "temperature_kelvin", "pressure_bar", "chem_potential_bulk"])
    writer.writeheader()
    writer.writerows(rows)

print(f"CSV gespeichert als: {output_csv} ({len(rows)} Einträge)")

if failed_structures:
    print("Fehlgeschlagene Strukturen:")
    for f in failed_structures:
        print(f" - {f['structure_name']}: {f['error']}")

CSV gespeichert als: /Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/bulk_potentials.csv (78128 Einträge)
